# Use the `DockerImage` class to dockerize a python function

The cloudknot `Knot` class creates its own docker image. For most users, it should work well "out of the box." For very complicated programs, you may need to edit the Dockerfile that cloudknot creates for your function. In that case, you can use the `DockerImage` class to dockerize your python function, make any edits that you need to make, and then send that `DockerImage` instance to a `Knot` using `Knot(..., docker_image=<your customized DockerImage instance>, ...)`.

In [1]:
import logging
import uuid
import cloudknot as ck

Set the logging level to print loging statements in the notebook

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Change this to logging.DEBUG if you want more verbose output

## Initialization

Create a test function, `test_func`, that `DockerImage` will dockerize. Sprinkle a bunch of import statements everywhere.

In [3]:
def test_func(name=None, *, no_capitalize=False):
    """Test function for unit testing of cloudknot.DockerReqs

    Import statements of various formats are deliberately scattered
    throughout the function to test the pipreqs components of
    clouknot.DockerReqs
    """
    import AFQ
    import sys
    import boto3.ec2
    if name:
        from docker import api
        from os.path import join

        if not no_capitalize:
            import pytest as pt
            import h5py.utils as h5utils

            name = name.title()

        return 'Hello {0}!'.format(name)

    from six import binary_type as bt
    from dask.base import curry as dbc

    return 'Hello world!'

Create a `DockerImage` instance from the test function. Since `AFQ` is not available via `pip install`, give the `DockerImage` its github address to install from github.

In [4]:
image = ck.DockerImage(func=test_func,
                       base_image="python:3.7",
                       github_installs=(
                           'https://github.com/yeatmanlab/pyAFQ.git',
                       ))
print(image.name)
print(image.build_path)
print(image.req_path)
print(image.docker_path)
print(image.script_path)
print(image.pip_imports)
print(image.github_installs)
print(image.missing_imports)
print(image.username)
print(image.func)

test_func
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_4vkenxtc
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_4vkenxtc/requirements.txt
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_4vkenxtc/Dockerfile
/Users/Adam/code/projects/cloudknot/examples/cloudknot_docker_test_func_4vkenxtc/test_func.py
[{'name': 'boto3', 'version': '1.4.7'}, {'name': 'cloudpickle', 'version': '0.4.1'}, {'name': 'dask', 'version': '0.15.4'}, {'name': 'docker', 'version': '2.5.1'}, {'name': 'h5py', 'version': '2.7.1'}, {'name': 'pytest', 'version': '3.2.3'}, {'name': 'six', 'version': '1.11.0'}]
['https://github.com/yeatmanlab/pyAFQ.git']
['AFQ']
cloudknot-user
<function test_func at 0x113e36d08>


If you didn't supply the github address, or if your dependencies require a bit more work to install, simply head to the Dockerfile at `image.docker_path` and edit the Dockerfile by hand with your more complicated install. Then resume with the following commands.

## Building the image

On `__init__`, `DockerImage` just creates all of the pre-requisite files: a Dockerfile, requirements.txt, and a CLI script. Since we haven't built or pushed this docker image yet, some of the parameters are empty or `None`.

In [5]:
print(image.images)
print(image.repo_uri)

[]
None


Let's actually build this image locally. You'll need Docker running on your machine for this to work. We'll pass in a list of tags. You can use any tag you want, as long as it isn't "latest."

In [6]:
image.build(tags=['example'])

Now that we've built the image, the `image_name` and `tags` parameters should be set. Since we haven't pushed the image to a repository, the `repo_uri` parameter is still `None`.

In [7]:
print(image.images)
print(image.repo_uri)

[{'name': 'cloudknot/test_func', 'tag': 'example'}]
None


## Pushing to a remote repository (by specifying a `DockerRepo` instance)

`DockerImage` instances know how to interact with `DockerRepo` instances. So we could create a new repo using cloudknot's `DockerRepo` class and pass that instance to the `image.push()` method.

In [8]:
repo = ck.aws.DockerRepo(name=str(uuid.uuid4()))
image.push(repo=repo)

Get the URI of the remote repo

In [9]:
print(image.images)
print(image.repo_uri)

[{'name': 'cloudknot/test_func', 'tag': 'example'}]
455598791984.dkr.ecr.us-east-1.amazonaws.com/4852adc8-89df-4ddd-a434-e4154677bdd7:example


Clean up after ourselves using the `clobber` method. This will remove the associated local docker images and the pre-requisite files. By design, it will not delete the remote repository or remote image. This must be done by calling the `DockerRepo.clobber()` method.

In [10]:
image.clobber()
repo.clobber()